In [28]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# Scraping data related to the Mission to Mars

In [3]:
# Create connect and create database in Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Create 'marsDB' in Mongo
db = client.marsDB

#### Scrape NASA Mars News

In [4]:
# Look at news page for Mars from NASA
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
# Create a Beautiful Soup object
soup = bs(html, 'html.parser')

In [5]:
# Collect latest news title
news_title = soup.find_all('div', class_="content_title")
print(news_title[0].text)

InSight Is the Newest Mars Weather Service


In [6]:
# Collect latest new paragraph
news_p = soup.select_one('ul.item_list li.slide div.article_teaser_body').text
print(news_p)

By collecting data around the clock, NASA's lander will provide unique science about the Martian surface.


#### Scrape JPL Mars Space Images - Featured Image

In [7]:
# URL of Mars Images website
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_url)

img_html = browser.html
# Create a Beautiful Soup object 
img_soup = bs(img_html, 'html.parser')

In [12]:
# Collect current featured Mars image
first_img = img_soup.select_one('div.image_and_description_container div.img img.thumb')
first_img_src = first_img['src']

base_img_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize'
featured_img_url = base_img_url + first_img_src

# Print compiled url
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/spaceimages/images/wallpaper/PIA23040-640x350.jpg


#### Scrape Latest Mars Weather Tweet

In [18]:
# URL of Mars Weather twitter page
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)

twitter_html = browser.html

# Create a Beautiful Soup object
tweet_soup = bs(twitter_html, 'html.parser')

In [25]:
# Save text from mars weather latest tweet
mars_weather = tweet_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather)

InSight sol 84 (2019-02-20) low -95.1ºC (-139.2ºF) high -13.2ºC (8.3ºF)
winds from the SW at 4.1 m/s (9.3 mph) gusting to 10.8 m/s (24.2 mph)pic.twitter.com/WlR4gr8gpC


#### Scrape Mars Facts

In [29]:
# URL of Space Facts - Mars Facts
facts_url = "https://space-facts.com/mars/"

facts_table = pd.read_html(facts_url)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]